Fine-tuning best T5 Transformer 🤖
-----------------------------------

In this notebook, we will continue the fine-tuning of T5 transformer on the new extracted sentences from the book **Grammaire de Wolof Moderne** without considering the definitions. We provide, bellow, the main evaluation figures, obtained from the hyperparameter search step. We will evaluate the training on the validation dataset.

- Parallel coordinates from panel:

- Parameter importance char: 
[t5_v3_importance](https://wandb.ai/oumar-kane-team/small-t5-cross-fw-translation-bayes-hpsearch-v3/reports/undefined-23-05-16-10-36-17---Vmlldzo0Mzc4NDY0?accessToken=eyaiyrid0qz1zg2jkq3fc65biw53084dpfitbi0dgonq6mweupw6kgjml9d2nv1w)

We can see in the above chart that the batch is the most important parameter with a negative correlation with the BLEU score (meaning that a lower batch size is better). Next, we the probability of modifying a character in the french corpus is also important and a high probability provide a better BLEU score.  

In [1]:
# let us import all necessary libraries
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5TokenizerFast, set_seed, AdamW, get_linear_schedule_with_warmup, T5ForConditionalGeneration,\
    get_cosine_schedule_with_warmup, Adafactor
from wolof_translate.utils.sent_transformers import TransformerSequences
from wolof_translate.utils.improvements.end_marks import add_end_mark # added
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer
from torch.utils.data import Dataset, DataLoader, random_split
from wolof_translate.data.dataset_v4 import SentenceDataset # v2 -> v3 -> v4
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import _LRScheduler
# from custom_rnn.utils.kwargs import Kwargs
from torch.nn.utils.rnn import pad_sequence
from plotly.subplots import make_subplots
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from torch.nn import functional as F
import plotly.graph_objects as go
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
import pytorch_lightning as lt
from tqdm import tqdm, trange
from functools import partial
from torch.nn import utils
from copy import deepcopy
from torch import optim
from typing import *
from torch import nn
import pandas as pd
import numpy as np
import itertools
import evaluate
import random
import string
import shutil
import wandb
import torch
import json
import copy
import os

# add seed for everything
lt.seed_everything(0)

os.environ["WANDB_DISABLED"] = "true"

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


## French to wolof

### Configure dataset 🔠

In [2]:
# recuperate the tokenizer from a json file
tokenizer = T5TokenizerFast(tokenizer_file=f"wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v3.json")


In [3]:
def recuperate_datasets(wf_char_p: float, wf_word_p: float, max_len: int, end_mark_opt: int):

  # Let us recuperate the end_mark adding option
  if end_mark_opt == 1:
    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=wf_char_p, aug_word_p=wf_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space)

  else:
    
    if end_mark_opt == 2:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!', replace = True)
    
    elif end_mark_opt == 3:

      end_mark_fn = partial(add_end_mark)
    
    elif end_mark_opt == 4:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!')

    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=wf_char_p, aug_word_p=wf_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)

  # Recuperate the train dataset
  train_dataset_aug = SentenceDataset(f"data/extractions/new_data/train_set.csv",
                                        tokenizer,
                                        truncation = True, max_len=max_len,
                                        corpus_1 = 'wolof',
                                        corpus_2='french',
                                        cp1_transformer = fr_augmentation_1,
                                        cp2_transformer = fr_augmentation_2
                                        )

  # Recuperate the valid dataset
  valid_dataset = SentenceDataset(f"data/extractions/new_data/valid_set.csv",
                                        tokenizer, max_len=max_len,
                                        corpus_1='wolof',
                                        corpus_2='french',
                                        cp1_transformer = fr_augmentation_2,
                                        cp2_transformer = fr_augmentation_2,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Searching for the best parameters 🕖

In [4]:
from wolof_translate.models.transformers.optimization import TransformerScheduler
from wolof_translate.trainers.transformer_trainer import ModelRunner
from wolof_translate.utils.evaluation import TranslationEvaluation
from wolof_translate.models.transformers.main import Transformer
from wolof_translate.utils.split_with_valid import split_data


-------------

### --- Wandb V3

In [11]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.03752442689238307,
    'fr_word_p': 0.4977518161400147,
    'learning_rate': 0.007500941460607961,
    'weight_decay': 0.01706989180286961,
    'batch_size': 16,
    'warmup_ratio': 0.0,
    'max_epoch': 578,
    'epochs': 58,
    'mid_epoch': 58,
    'max_len': 35,
    'end_mark': 3,
    'bleu': 10.4727,
    'model_dir': 'data/checkpoints/wf_t5_small_custom_train_v3_checkpoints/',
    'new_model_dir': 'data/checkpoints/t5_small_custom_train_results_wf_v3/'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = 3, evaluation = evaluation, optimizer = Adafactor)

# split the data
split_data(config['random_state'])

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'],
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'warmup_init': False,
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_linear_schedule_with_warmup,
                # lr_scheduler=get_cosine_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_wf"
                )

# We will from checkpoints so let us the model
# trainer.load(config['model_dir'], load_best=True) # Only for the first loading
trainer.load(config['new_model_dir'], load_best=True)

        

### --- Linear

In [10]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/53 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 6: 
{Learning rate: [0.007435243335790785]}


Test batch number 11: 100%|██████████| 11/11 [00:02<00:00,  3.81batches/s]



Metrics: {'train_loss': 0.2671666884863818, 'test_loss': 0.8340660929679871, 'bleu': 7.6334, 'gen_len': 8.5864}




  2%|▏         | 1/53 [00:44<38:10, 44.05s/it]

...

 98%|█████████▊| 52/53 [21:56<00:24, 24.19s/it]

For epoch 58: 
{Learning rate: [0.006751982837692153]}


Test batch number 11: 100%|██████████| 11/11 [00:02<00:00,  3.92batches/s]



Metrics: {'train_loss': 0.020747645229193163, 'test_loss': 0.9395695545456626, 'bleu': 20.8684, 'gen_len': 8.679}




100%|██████████| 53/53 [22:20<00:00, 25.29s/it]


### --- Cosine

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

### Predictions and Evaluation

In [12]:
# initialize the transformation sequence
end_mark_fn = partial(add_end_mark)
fr_augmentation = TransformerSequences(remove_mark_space, delete_guillemet_space, end_mark_fn)

# let us get the test set
test_dataset = SentenceDataset(f"data/extractions/new_data/test_set.csv",
                                        corpus_1='wolof',
                                        corpus_2='french',
                                        tokenizer = tokenizer,
                                        cp1_transformer = fr_augmentation,
                                        cp2_transformer = fr_augmentation,
                                        truncation = True)

Let us make the evaluation and print the predicted sentences.

In [13]:
# evaluation with test set
df_ft_to_wf = trainer.evaluate(test_dataset)

Evaluation batch number 11: 100%|██████████| 11/11 [00:04<00:00,  2.39batches/s]


predictions_: [[0, 19, 10, 38, 36, 22, 116, 10, 16, 38, 36, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 35, 372, 21, 325, 7, 210, 22, 3, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 154, 716, 65, 10, 128, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 222, 169, 739, 18, 138, 23, 63, 111, 1489, 142, 3, 61, 209, 138, 23, 702, 620, 281, 406, 24, 198, 7, 63, 1281, 4, 1], [0, 33, 57, 30, 38, 966, 14, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 190, 32, 63, 570, 545, 16, 30, 195, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 35, 85, 30, 66, 607, 7, 21, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 42, 76, 1040, 83, 500, 24, 99, 366, 360, 503, 16, 7, 383, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 986, 8, 22, 27, 9, 589, 17, 3, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 392, 1495, 1803, 250, 7, 276, 7, 3, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 574, 8, 107, 587, 13

In [14]:
df_ft_to_wf[1].tail(10)

,original_sentences,translations,predictions
151,Ñii dañu demul xanaa!,Ceux-ci ne sont peut-être pas partis!,Ceux-ci ne partent peut-être pas!
152,Keneen a ŋgi.,En voilà un autre.,En voilà un.
153,"Bon nag, maa ŋgi dem.","Dans ce cas, je pars.",Surtout rentrez chez vous.
154,Ana ŋgoor?,Où est NGor?,Où est-il?
155,"Defe naa du kookuu, du kookee.","Je crois que ce n'est pas celui-ci, ce n'est p...","Je ne veux pas, il verra."
156,Suñu xarit la!,C'est leur ami!,C'est notre ami!
157,Xale bi ne ŋgeen dugg mu gis leen.,L'enfant dit d'entrer pour qu'il vous voie.,Cet autre qui as-tu vu?
158,Gis naa góor gi doon wax te toogoon ci suuf.,J'ai vu cet homme qui parlait et qui était ass...,J'ai vu cet homme et lion ne cohabitent.
159,Dimëlé leen ŋgir ñooñe!,Aide-les à cause de ceux-là!,Sois quelqu'un de studieux!
160,Mi ŋgiy laaj ndax demkoon ŋga.,Il demande si tu aurais été.,Il est là ces livres là-bas.


In [16]:
# let us display 100 samples
pd.options.display.max_rows = 100
df_ft_to_wf[1].sample(100)

,original_sentences,translations,predictions
46,Dafa doon nitu dëgg.,C'est un homme de vérité.,Il travaillait.
76,Waxtaan ak kenn kan?,Converser avec lequel?,Personne n'est donc parti?
98,Waaji nee.,Mi ŋgi ci biti.,Ta mère dit qu'elle reviendra si tu vas jusqu'...
128,Ana góor gi?,Où est l'homme?,Où est le Monsieur?
15,Góor gi nee na soo demee mi ŋgi fi.,L'homme dit qu'il serait peut-être là.,L'homme affient que les enfants y eussent été.
72,Gisoon naa xar yi yépp.,J'ai déjà vu tous les moutons.,J'ai vu les enfants sauf toi.
112,Loo gis?,Qu'as-tu vu?,Qui as-tu vu?
110,"Góor googule ñëwóon, mi ŋgi!","Cet homme qui vint autrefois, le voilà!",Cet homme le voilà!
123,Man xar moo réer?,Quel moutons est égaré?,Quel moutons est égaré?
2,Liggéeykat yi man ag yaw la.,Les travailleurs c'est toi et moi.,Les travailleurs c'est toi.
